In [1]:
import csv
import pandas
import numpy as np
import os.path as osp
from pandas import DataFrame
from general_tools.in_out.basics import files_in_subdirs
import matplotlib.pylab as plt

In [ ]:
%matplotlib inline

In [2]:
top_in_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Language/'
dialogues_dir = osp.join(top_in_dir, 'pilot_chair_data/chatMessage')
dialogue_files = [f for f in files_in_subdirs(dialogues_dir, '.csv')]

In [15]:
class Language_Game_Data_Loader(): 
        
    @staticmethod
    def load_dialogues_of_game(game_file):
        special_tokens = {'speaker': ' SpEaKeR: ', 'listener': ' lIsTeNeR: '}    
            
        # Hard-coded incides of interesting quantities.
        triplet_id = 4
        human_role = 5
        sentence_pos = 6
        trial_to_data = {}

        with open(osp.join(top_in_dir, game_file), 'rb') as csvfile:
            reader = csv.reader(csvfile, delimiter='\t')

            for i, row in enumerate(reader):
                if i == 0:                    
                    continue
                    
                tr_id = row[triplet_id].split('_')
                
                if tr_id[0] not in ['far', 'split', 'close'] or not tr_id[1].isdigit():
                    print 'Entry without proper triplet_id was found.'
                    continue

                tr_id = tr_id[0] + '_' + tr_id[1]    
                role = special_tokens[row[human_role]]
                sentence = row[sentence_pos]

                if tr_id in trial_to_data:
                    trial_to_data[tr_id]['dialogue'] += role + sentence
                else:
                    trial_to_data[tr_id] = dict()
                    trial_to_data[tr_id]['dialogue'] = role + sentence
            return trial_to_data
        
    @staticmethod
    def load_selections_of_game(game_file):
        with open(osp.join(top_in_dir, game_file), 'rb') as csvfile:
            reader = csv.reader(csvfile, delimiter='\t')
            for i, row in enumerate(reader):
                if i == 0:                    
                    continue
                print row[15]
    
    @staticmethod
    def _triplet_to_model_map(map_file):
        res = dict()
        with open (map_file, 'r') as fin:
            for line in fin:
                tokens = line.split()
                res[tokens[0]] = (tokens[1], tokens[2], tokens[3])
            return res

    @staticmethod    
    def triplet_to_model_map(close_map_file, far_map_file, split_map_file):
        triplet_to_model_map = {
        'close': Language_Game_Data_Loader._triplet_to_model_map(close_map_file),
        'far': Language_Game_Data_Loader._triplet_to_model_map(far_map_file),
        'split': Language_Game_Data_Loader._triplet_to_model_map(split_map_file)}
        return triplet_to_model_map

        
def load_glove_pretrained_model(glove_file):
    print "Loading glove model."
    embedding = dict()
    with open(glove_file, 'r') as f_in:
        for line in f_in:
            s_line = line.split()
            word = s_line[0]
            w_embedding = np.array([float(val) for val in s_line[1:]], dtype=np.float32)
            embedding[word] = w_embedding
    print "Done.", len(embedding), " words loaded!"
    return embedding

In [13]:
close_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/closest_nn/data/model_names_of_triplets.txt'
split_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/one_far/data/model_names_of_triplets.txt'
far_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/both_far/data/model_names_of_triplets.txt'

In [5]:
triplet_to_model_names = Language_Game_Data_Loader.triplet_to_model_map(close_map_file, far_map_file, split_map_file)

In [16]:
language_data = []
for f in dialogue_files:
    language_data.append(Language_Game_Data_Loader.load_dialogues_of_game(f))

Entry without proper triplet_id was found.
Entry without proper triplet_id was found.
Entry without proper triplet_id was found.
Entry without proper triplet_id was found.
Entry without proper triplet_id was found.
Entry without proper triplet_id was found.


In [7]:
geo_embedding = pandas.read_pickle('/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_chair_embeddings.pkl')
word_emb_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Language/glove.6B/glove.6B.100d.txt'
word_dict = load_glove_pretrained_model(word_emb_file)

Loading glove model.
Done. 400000  words loaded!


In [8]:
def embedding_dictionary_to_matrix(in_dict):
    return np.array(in_dict.values())

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

replace_dic = {'!': ' ', ',': ' ', '?': ' ', '.': ' ', ';': ' ', '(': ' ', ')': ' ', '/': ' ',
               '&': 'and', '"': ' ', '-': ' ', '=': ' '
              }

In [114]:
# Deal with "speaker:", "listener:"
# misspelled words
# dentist's chair

In [215]:
# Use tf.nn.embedding_lookup instead and map words to integers

words = set()
words_embedded = []
train_data = dict()

for i, game in enumerate(language_data):
    for triplet_id, dialogues in game.iteritems():
        triplet_condition, triplet_num = triplet_id.split('_')
        a, b, c = triplet_to_model_names[triplet_condition][triplet_num]
        triplet_geo_emb = geo_embedding.loc[[a, b, c]].as_matrix().T
        index = np.arange(3)
        np.random.shuffle(index)
        triplet_geo_emb = triplet_geo_emb[:, index]
        label = np.where(index==0)[0][0]
        
        triplet_word_seq = []        
        for w in replace_all(dialogues['dialogue'], replace_dic).split():
            wl = w.lower()
            if wl in word_dict:
                triplet_word_seq.append(word_dict[wl])
                words.add(wl)                
            else: #TODO deal with uknown words
#                 'unk' in word_dict
                pass 
        
        if len(triplet_word_seq) >= 1:
            train_data[(i, triplet_id)] = {'label': label}
            train_data[(i, triplet_id)]['word_feats'] =  np.array(triplet_word_seq)
            train_data[(i, triplet_id)]['geo_feats'] = triplet_geo_emb.copy()

In [227]:
set(key[1].split('_')[1] for key in train_data.keys())


{'1',
 '10',
 '101',
 '102',
 '106',
 '107',
 '109',
 '11',
 '111',
 '113',
 '114',
 '116',
 '117',
 '118',
 '12',
 '120',
 '121',
 '122',
 '123',
 '128',
 '129',
 '13',
 '130',
 '133',
 '134',
 '135',
 '138',
 '14',
 '16',
 '17',
 '18',
 '2',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '35',
 '38',
 '39',
 '4',
 '48',
 '5',
 '50',
 '51',
 '57',
 '58',
 '59',
 '6',
 '60',
 '62',
 '64',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '74',
 '79',
 '8',
 '82',
 '84',
 '86',
 '89',
 '9',
 '91',
 '95',
 '98'}

In [205]:
n_triplets = len(train_data)
max_seq_len = 20
example = train_data[train_data.keys()[0]]
word_data = np.zeros(shape=((n_triplets, max_seq_len,example['word_feats'].shape[1])))

all_seq_len = []

for i, key in enumerate(train_data):
    datum = train_data[key]['word_feats']       
    n_steps = datum.shape[0]
    all_seq_len.append(n_steps)        
    pad_many = max_seq_len-n_steps
    if pad_many > 0:
        word_data[i] = np.pad(datum, ((0, pad_many), (0,0)) , 'constant')
    elif pad_many < 0:
        word_data[i] = train_data[key]['word_feats'][:max_seq_len,:]
    

geo_data = np.zeros(shape=(n_triplets, np.prod(example['geo_feats'].shape)) )
label_data = np.zeros((n_triplets, 3), dtype=np.int32)
for i, key in enumerate(train_data):
    datum = train_data[key]['geo_feats']       
    geo_data[i] = np.hstack(datum)
    label = train_data[key]['label']
    label_data[i][label] = 1
    

In [206]:
class GeoWordsDataSet(object):

    def __init__(self, words, geometries, labels, use_copies=True, init_shuffle=False):
        self.num_examples = words.shape[0]
        assert(self.num_examples == labels.shape[0])
        assert(self.num_examples == geometries.shape[0])
        
        assert( use_copies )
        assert(init_shuffle==False)
        
        self.words = words.copy()        
        self.geometries = geometries.copy()
        self.labels = labels.copy()

        self.epochs_completed = 0
        self._index_in_epoch = 0
        if init_shuffle:
            self.shuffle_data()
            
    def shuffle_data(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        perm = np.arange(self.num_examples)
        np.random.shuffle(perm)
        self.words = self.words[perm]
        self.geometries = self.geometries[perm]
        self.labels = self.labels[perm]
        return self

    def next_batch(self, batch_size, seed=None):
        '''Return the next batch_size examples from this data set.
        '''
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self.num_examples:
            self.epochs_completed += 1  # Finished epoch.
            self.shuffle_data(seed)
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
        end = self._index_in_epoch

        return self.words[start:end], self.geometries[start:end], self.labels[start:end]

In [207]:
gw_data = GeoWordsDataSet(words=word_data, geometries=geo_data, labels=label_data)

In [208]:
from general_tools.in_out.basics import pickle_data, unpickle_data

In [209]:
pickle_data('first_pilot_data.pkl', gw_data)

In [210]:
lala = unpickle_data('first_pilot_data.pkl').next()

In [214]:
lala.next_batch(1)[2]

array([[0, 0, 1]], dtype=int32)

In [101]:
# map numbers to language
# typos


# sqare -> square 
# sqiggle -> squiggly
# shorest -> shortest
# pilow -> pillow